# Scraper to get hockey players' height and weight

To limit the need for manual research for each player when watching the videos \
Azilis EVEN

In [1]:
import requests
from bs4 import BeautifulSoup
import json
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl import worksheet
import os

In [2]:
###### TO FILL IN #######

# Player jersey number
jerseyNb = 7

# Season (in the "2017-2018" format)
season = "2017-2018"

# Choose the club in the following list
# 2/brynas-if 3/djurgardens-if 4/farjestad-bk 5/hv71 6/linkoping-hc 7/lulea-hf 8/malmo-redhawks 12/frolunda-hc 
# 22/skelleftea-aik 28/leksands-if 32/rogle-bk 36/orebro-hk 339/vaxjo-lakers-hc
# If the club is not in the list, check https://www.eliteprospects.com/league/shl or https://www.eliteprospects.com/league/hockeyallsvenskan
club = "7/lulea-hf"

# Path of the spreadsheet to save the info, if needed
pathExcel = "players-bmi-info.xlsx"

In [3]:
# Get page source code and parse it
URL = "https://www.eliteprospects.com/team/" + club + "/" + season
print(URL)
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

https://www.eliteprospects.com/team/7/lulea-hf/2017-2018


In [4]:
# Get the info for the BMI calculations

results = soup.find(id="__NEXT_DATA__")
resultsTXT = results.text
resultsJSON = json.loads(resultsTXT)
playersRoster = resultsJSON["props"]["pageProps"]["rosterList"]["tableData"]["edges"]
playerSelected = list(filter(lambda x:x["jerseyNumber"]==jerseyNb,playersRoster))[0]
playerName = playerSelected["player"]["name"]
print(playerName)
playerHeight = playerSelected["player"]["height"]["metrics"]
print(str(playerHeight) + " cm")
playerWeight = playerSelected["player"]["weight"]["metrics"]
print(str(playerWeight) + " kg")

Jan Sandström
184 cm
89 kg


In [8]:
# Create/open the excel sheet and create the player info sheet if needed
# Then set this sheet as active sheet
if not os.path.isfile(pathExcel):
    workbook = Workbook()
    sheetPlayers = workbook.active
    sheetPlayers.title = 'players.info'
else:
    workbook = load_workbook(filename=pathExcel)
    if not 'players.info' in workbook.sheetnames:
        sheetPlayers = workbook.create_sheet('players.info')
    else:
        sheetPlayers = workbook['players.info']
workbook.save(filename=pathExcel)

# Add colum names if needed
if not sheetPlayers["A1"] == "season":
    sheetPlayers.insert_rows(idx=1)
    columns = ["season", "jerseyNb", "playerName", "club", "playerHeight", "playerWeight"]
    for i in range(len(columns)):
            cell=sheetPlayers.cell(row=1, column=i+1)
            cell.value=columns[i]
workbook.save(filename=pathExcel)

In [10]:
# Create new row for the player info
newRowLocation = sheetPlayers.max_row +1
sheetPlayers.append([season, jerseyNb, playerName, club.split("/")[1], playerHeight, playerWeight])
workbook.save(filename=pathExcel)